In [2]:
import os
import numpy as np
import shutil
import tensorflow as tf
from typing import Tuple

from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import ParameterGrid
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import ELU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

from tf_utils.dogscatsDataAdvanced import DOGSCATS

In [3]:
np.random.seed(0)
tf.random.set_seed(0)

In [4]:
LOGS_DIR = os.path.abspath('C:/Selbststudium/Udemy/Udemy_Tensorflow/logs/ACTIVATIONS/')
if not os.path.exists(LOGS_DIR):
    os.mkdir(LOGS_DIR)

In [5]:
def build_model(
    img_shape: Tuple[int, int, int],
    num_classes: int,
    optimizer: tf.keras.optimizers.Optimizer,
    learning_rate: float,
    filter_block_1: int,
    kernel_size_block_1: int,
    filter_block_2: int,
    kernel_size_block_2: int,
    filter_block_3: int,
    kernel_size_block_3: int,
    dense_layer_size: int,
    kernel_initializer: tf.keras.initializers.Initializer,
    activation_cls: tf.keras.layers.Activation
) -> Model:
    input_img = Input(shape=img_shape)

    x = Conv2D(
        filters=filter_block_1, 
        kernel_size=kernel_size_block_1, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(input_img)
    x = activation_cls(x)
    x = Conv2D(
        filters=filter_block_1, 
        kernel_size=kernel_size_block_1, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    x = activation_cls(x)
    x = MaxPool2D()(x)

    x = Conv2D(
        filters=filter_block_2, 
        kernel_size=kernel_size_block_2, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    x = activation_cls(x)
    x = Conv2D(
        filters=filter_block_2, 
        kernel_size=kernel_size_block_2, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    x = activation_cls(x)
    x = MaxPool2D()(x)

    x = Conv2D(
        filters=filter_block_3, 
        kernel_size=kernel_size_block_3, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    x = activation_cls(x)
    x = Conv2D(
        filters=filter_block_3, 
        kernel_size=kernel_size_block_3, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    x = activation_cls(x)
    x = MaxPool2D()(x)

    x = Flatten()(x)
    x = Dense(
        units=dense_layer_size, 
        kernel_initializer=kernel_initializer
    )(x)
    x = activation_cls(x)
    x = Dense(
        units=num_classes, 
        kernel_initializer=kernel_initializer
    )(x)
    y_pred = Activation('softmax')(x)

    # Jetzt muss noch ein Modell Objekt mit eben obiger Struktur erstellt werden
    model = Model(
        inputs = [input_img],
        outputs = [y_pred]
    )
    
    opt = optimizer(learning_rate=learning_rate)
    
    model.compile(
        loss='categorical_crossentropy', # wird bei Kategorie-Problemen mit mehr als 2 Klassen genommen
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

In [6]:
data = DOGSCATS()

train_dataset = data.get_train_set()
val_dataset = data.get_val_set()
test_dataset = data.get_test_set()

img_shape = data.img_shape
num_classes = data.num_classes

epochs = 40
batch_size = 128

# Hier nun die Parameter des Models der GridSearch eintragen
# param_grid={...} wird hier nicht mehr benötigt
# Best model params
optimizer = Adam
learning_rate = 0.001
filter_block1 = 32
kernel_size_block1 = 3
filter_block2 = 64
kernel_size_block2 = 3
filter_block3 = 128
kernel_size_block3 = 3
dense_layer_size = 128
kernel_initializer = "GlorotUniform"

In [7]:
# In diesem Video bzw. genau diesem Abschnitt werden verschiedene Activations getestet
activations = {
    "RELU": ReLU(),
    "LEAKY_RELU": LeakyReLU(),
    "ELU": ELU()
}

for activation_key in activations:
    activation_cls = activations[activation_key]

    model = build_model(
        img_shape,
        num_classes,
        optimizer,
        learning_rate,
        filter_block1,
        kernel_size_block1,
        filter_block2,
        kernel_size_block2,
        filter_block3,
        kernel_size_block3,
        dense_layer_size,
        kernel_initializer,
        activation_cls
    )
    model_log_dir = os.path.join(LOGS_DIR, f"ACTIVATION_{activation_key}")

    tb_callback = TensorBoard(
        log_dir=model_log_dir,
        histogram_freq=0,
        profile_batch=0,
        write_graph=0
    )

    model.fit(
        train_dataset,
        verbose=1,
        batch_size=batch_size,
        epochs=epochs,
        callbacks=[tb_callback],
        validation_data=val_dataset,
    )

Epoch 1/40
105/105 [==============================] - 17s 79ms/step - loss: 0.7545 - accuracy: 0.5066 - val_loss: 0.6876 - val_accuracy: 0.5439
Epoch 2/40
105/105 [==============================] - 7s 68ms/step - loss: 0.6624 - accuracy: 0.5987 - val_loss: 0.6476 - val_accuracy: 0.6594
Epoch 3/40
105/105 [==============================] - 7s 68ms/step - loss: 0.6104 - accuracy: 0.6723 - val_loss: 0.5702 - val_accuracy: 0.6980
Epoch 4/40
105/105 [==============================] - 7s 68ms/step - loss: 0.5503 - accuracy: 0.7257 - val_loss: 0.5115 - val_accuracy: 0.7531
Epoch 5/40
105/105 [==============================] - 7s 67ms/step - loss: 0.5070 - accuracy: 0.7559 - val_loss: 0.4979 - val_accuracy: 0.7691
Epoch 6/40
105/105 [==============================] - 7s 67ms/step - loss: 0.4642 - accuracy: 0.7815 - val_loss: 0.4614 - val_accuracy: 0.7811
Epoch 7/40
105/105 [==============================] - 7s 67ms/step - loss: 0.4362 - accuracy: 0.7970 - val_loss: 0.4453 - val_accuracy: 0.795

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\gutsc\anaconda3\envs\udemy_tensorflow\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\gutsc\AppData\Local\Temp/ipykernel_7448/3913789588.py", line 36, in <module>
    model.fit(
  File "C:\Users\gutsc\anaconda3\envs\udemy_tensorflow\lib\site-packages\keras\utils\traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\gutsc\anaconda3\envs\udemy_tensorflow\lib\site-packages\keras\engine\training.py", line 1252, in fit
    val_logs = self.evaluate(
  File "C:\Users\gutsc\anaconda3\envs\udemy_tensorflow\lib\site-packages\keras\utils\traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\gutsc\anaconda3\envs\udemy_tensorflow\lib\site-packages\keras\engine\training.py", line 1537, in evaluate
    tmp_logs = self.test_function(iterator)
  File "C:\Users\gutsc\anacond

In [ ]:
print(f"Test performance best model: {val_accuracy}")

Test performance best model: [0.9349769949913025, 0.8416833877563477]
